# Lattice Solver Tutorial

This tutorial using triqs lattice tools and the Hartree-Fock LatticeSolver for the case of a 2 degenerate bands with local Kanamori interactions at fixed density. At the end, we solve the same problem using the ImpuritySolver with the DMFT loop and arrive at the same solution.

We start by importing the required modules:

In [1]:
import numpy as np
from hartree_fock.solver import ImpuritySolver, LatticeSolver, flatten
from triqs.gf import *
from triqs.operators import *
from triqs.operators.util import *
from h5 import HDFArchive
from triqs.lattice.tight_binding import TBLattice
from triqs.sumk import *
from triqs.lattice import *
from scipy.optimize import brentq

Starting serial run at: 2022-04-27 12:08:37.534398


First we define a 2D tight binding lattice with nearest and next-nearest neighbor hopping:

In [2]:
t = np.array([[1, 0.2], [0.2, 1]])
tp = np.array([[0.1, 0.05], [0.05, 0.1]])
hoppings= {  (1,0)  : t,       
        (-1,0) :  t,     
        (0,1)  :  t,
        (0,-1) :  t,
        (1,1)  :  tp,
        (-1,-1):  tp,
        (1,-1) :  tp,
        (-1,1) :  tp}

TBL = TBLattice(units=[(1, 0, 0) , (0, 1, 0)], hoppings=hoppings, orbital_positions=[(0., 0., 0.)]*2)

Next we define the single particle dispersion h0_k and the other parameters to pass to the solver

In [3]:
nk = 10
beta = 40
U = 3
J = 0.5
gf_struct = [('up', 2), ('down', 2)]
N_target = 2

#create a kanamori interaction Hamiltonian
Umat, Upmat = U_matrix_kanamori(n_orb=2, U_int=U, J_hund=J)
h_int = h_int_kanamori(spin_names=['up', 'down'], orb_names=[0,1], U=Umat, Uprime=Upmat, J_hund=J, off_diag=True)

BL = BravaisLattice(units = [(1,0,0) , (0,1,0)])
BZ = BrillouinZone(BL)
mk = MeshBrZone(BZ, nk)
ekup = Gf(mesh=mk, target_shape=[2,2])
ekdn = Gf(mesh=mk, target_shape=[2,2])
ekup << TBL.fourier(mk)
ekdn << TBL.fourier(mk) 
h0_k = BlockGf(name_list=['up', 'down'], block_list=(ekup, ekdn)) #single particle dispersion

#symmetry function forcing two spins to be equal
def make_spins_equal(Sigma):
    Symmetrized_Sigma = Sigma.copy()
    Symmetrized_Sigma['up'] = 0.5*(Sigma['up']  + Sigma['down'])
    Symmetrized_Sigma['down'] = Symmetrized_Sigma['up']
    return Symmetrized_Sigma

Now we are ready to initialize the solver. For illustration purposes, we pass a symmetry function forcing the spins to be equal, even though it is not necessary in this case.

In [4]:
S = LatticeSolver(h0_k=h0_k, h_int=h_int, gf_struct=gf_struct, beta=beta, symmetries=[make_spins_equal])

and finally we call the solve method at fixed density of 2 electrons per site:

In [5]:
S.solve(N_target=N_target)


╔╦╗╦═╗╦╔═╗ ╔═╗  ┬ ┬┌─┐
 ║ ╠╦╝║║═╬╗╚═╗  ├─┤├┤ 
 ╩ ╩╚═╩╚═╝╚╚═╝  ┴ ┴└  
TRIQS: Hartree-Fock solver

Running Lattice Solver at fixed density of 2.0000
beta = 40.0000
h_int =
1.5*c_dag('down',0)*c_dag('down',1)*c('down',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',0)*c('up',1)*c('down',1) + 3*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0) + 2*c_dag('down',0)*c_dag('up',1)*c('up',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',1)*c('up',0)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',0)*c('up',1)*c('down',0) + 2*c_dag('down',1)*c_dag('up',0)*c('up',0)*c('down',1) + 3*c_dag('down',1)*c_dag('up',1)*c('up',1)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',1)*c('up',0)*c('down',0) + 1.5*c_dag('up',0)*c_dag('up',1)*c('up',1)*c('up',0)
mode: self-consistent
Including Fock terms:
True
Self Consistent Hartree-Fock converged successfully
Sigma_HF['up'] =
[[ 3.25 +0.j -0.012+0.j]
 [-0.012+0.j  3.25 +0.j]]
Sigma_HF['down'] =
[[ 3.25 +0.j -0.012+0.j]
 [-0.012+0.j  3.25 +0.j]]


## DMFT Loop with Impurity Solver:

We can arrive at the same solution of the lattice solver using a DMFT loop and impurity solver 

In [6]:
SK = SumkDiscreteFromLattice(lattice=TBL, n_points=nk)
sigma = GfImFreq(beta=beta, n_points=1025, target_shape=[2,2])
Sigma = BlockGf(name_list=['up', 'down'], block_list=(sigma,sigma), make_copies=True)
Gloc = Sigma.copy()
N_target = 2
mu = 0
S_imp = ImpuritySolver(h_int=h_int, gf_struct=gf_struct, beta=beta, n_iw=1025)

converged = False
while not converged:
    for name, bl in gf_struct:
        Sigma[name] << S.Sigma_HF[name]
    mu = brentq(lambda mu: SK(mu=mu, Sigma=Sigma).total_density().real - N_target, -5, 5)
    print('mu = ', mu)
    Gloc << SK(mu=mu, Sigma=Sigma)
    S_imp.G0_iw << inverse(inverse(Gloc) + Sigma)
    Sigma_old = S_imp.Sigma_HF.copy()
    S_imp.solve()
    if np.allclose(flatten(Sigma_old), flatten(S_imp.Sigma_HF), rtol=0, atol=1e-4):
        converged = True

mu =  3.1379971378941245

╔╦╗╦═╗╦╔═╗ ╔═╗  ┬ ┬┌─┐
 ║ ╠╦╝║║═╬╗╚═╗  ├─┤├┤ 
 ╩ ╩╚═╩╚═╝╚╚═╝  ┴ ┴└  
TRIQS: Hartree-Fock solver

Running Impurity Solver
beta = 40.0000
h_int =
1.5*c_dag('down',0)*c_dag('down',1)*c('down',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',0)*c('up',1)*c('down',1) + 3*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0) + 2*c_dag('down',0)*c_dag('up',1)*c('up',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',1)*c('up',0)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',0)*c('up',1)*c('down',0) + 2*c_dag('down',1)*c_dag('up',0)*c('up',0)*c('down',1) + 3*c_dag('down',1)*c_dag('up',1)*c('up',1)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',1)*c('up',0)*c('down',0) + 1.5*c_dag('up',0)*c_dag('up',1)*c('up',1)*c('up',0)
mode: self-consistent
Including Fock terms:
True
Self Consistent Hartree-Fock converged successfully
Sigma_HF['up'] =
[[ 3.25 +0.j -0.012+0.j]
 [-0.012+0.j  3.25 +0.j]]
Sigma_HF['down'] =
[[ 3.25 +0.j -0.012+0.j]
 [-0.012+0.j  3.25 +0.j]]
mu =  3.1379971378941

/usr/lib/python3/dist-packages/scipy/optimize/nonlin.py:475: RuntimeWarning: invalid value encountered in double_scalars
  and dx_norm/self.x_rtol <= x_norm))


mu =  3.1379971378941245

╔╦╗╦═╗╦╔═╗ ╔═╗  ┬ ┬┌─┐
 ║ ╠╦╝║║═╬╗╚═╗  ├─┤├┤ 
 ╩ ╩╚═╩╚═╝╚╚═╝  ┴ ┴└  
TRIQS: Hartree-Fock solver

Running Impurity Solver
beta = 40.0000
h_int =
1.5*c_dag('down',0)*c_dag('down',1)*c('down',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',0)*c('up',1)*c('down',1) + 3*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0) + 2*c_dag('down',0)*c_dag('up',1)*c('up',1)*c('down',0) + 0.5*c_dag('down',0)*c_dag('up',1)*c('up',0)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',0)*c('up',1)*c('down',0) + 2*c_dag('down',1)*c_dag('up',0)*c('up',0)*c('down',1) + 3*c_dag('down',1)*c_dag('up',1)*c('up',1)*c('down',1) + 0.5*c_dag('down',1)*c_dag('up',1)*c('up',0)*c('down',0) + 1.5*c_dag('up',0)*c_dag('up',1)*c('up',1)*c('up',0)
mode: self-consistent
Including Fock terms:
True
Self Consistent Hartree-Fock converged successfully
Sigma_HF['up'] =
[[2.6  +0.j 0.153+0.j]
 [0.153+0.j 2.6  +0.j]]
Sigma_HF['down'] =
[[ 3.956+0.j -0.17 +0.j]
 [-0.17 +0.j  3.956+0.j]]
mu =  3.1379971378941245
